<a href="https://colab.research.google.com/github/DanielaPosadas/SQL-Mystery/blob/main/Sql-Murder-Mystery/SQL_Murder_Mystery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/icarito/UPSK-SQL001-SQLZoo-murder/blob/sql-murder/Sql-Murder-Mystery/SQL_Murder_Mystery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Murder Mystery!
Adapted By: Martin Arroyo

![Detective making connections between points](https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/sleuth.png?raw=1)

**Credit**

This material was adapted from the [SQL Murder Mystery by Knight Lab](https://mystery.knightlab.com/) under [Creative Commons CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/). The SQL Murder Mystery was originally created by [Joon Park](https://twitter.com/joonparkmusic) and [Cathy He](https://twitter.com/Cathy_MeiyingHe) while they were Knight Lab fellows. See the [GitHub repository](https://github.com/NUKnightLab/sql-mysteries) for more information.

## Escenario

¡Se ha cometido un crimen y los detectives necesitan tu ayuda! Te dieron el informe de la escena del crimen, pero de alguna manera lo perdiste. Recuerdas vagamente que el crimen fue un `murder` _(asesinato)_ que ocurrió en algún momento el 15 de Enero de 2018 y que tuvo lugar en `SQL City`. Depende de ti averiguar quién es el asesino usando solo tus habilidades en SQL y tu ingenio. Te proporcionan una conexión a la base de datos del Departamento de Policía, que tiene todas las pistas que necesitarás para atrapar al asesino.

Usa las habilidades que desarrollaste en SQL 101 y 102, junto con cualquier recurso que desees, para resolver el `SQL Murder Mystery`!"

## Conéctate a la base de datos del Departamento de Policía

Para comenzar y ejecutar tus consultas, presiona play en la celda de abajo para conectarte a la base de datos del Departamento de Policía.

Para ejecutar consultas, crea una nueva celda de `Code` y escribe `%%sql` en la parte superior. Luego puedes escribir tus consultas debajo. Ve el ejemplo a continuación:
```python
%%sql

SELECT *
FROM table

```

In [9]:
%%capture
# @title Press Play { display-mode: "form" }
# Install `teachdb` and `coop_grader`
print("Installing `teachdb` and its dependencies...")
!pip install --quiet --upgrade git+https://github.com/freestackinitiative/teachingdb.git git+https://github.com/martinmarroyo/coop_grader.git
print("Successfully installed `teachdb`")
import pandas as pd
from teachdb.teachdb import connect_teachdb
from coop_grader.sql_murder_mystery.check_suspect import check_suspect
# Set configurations for notebook
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 99)
# Load data
con = connect_teachdb(database="sql_murder_mystery")

%sql con

## Descubriendo tablas en la base de datos

Comenzamos nuestra búsqueda para encontrar al asesino explorando la base de datos del Departamento de Policía. Pero aún no has visto la base de datos y no sabes cuáles son las tablas, así que, ¿cómo sabes qué buscar?

Afortunadamente, la mayoría de los sistemas de gestión de bases de datos relacionales tienen esta información almacenada en un lugar donde puedes consultarla. Muy a menudo, se utiliza un esquema especial conocido como [`information_schema`](https://en.wikipedia.org/wiki/Information_schema) para almacenar información sobre las tablas y columnas en tu base de datos (también conocido como metadata). La base de datos del Departamento de Policía tiene un information schema, con la vista `tables` que te muestra qué tablas están disponibles, y la vista `columns` que te muestra todas las columnas de cada tabla y sus tipos de datos.

### Listando todas las tablas en la base de datos de la Policía

#### Revisa la [descripción de `information_schema.tables`](https://duckdb.org/docs/sql/information_schema.html#tables-and-views)

Primero, veremos todas las tablas disponibles para nosotros consultando la vista `information_schema.tables`. Te daremos la primera consulta para empezar, pero de aquí en adelante tendrás que idear las consultas restantes utilizando tu conocimiento de SQL y tu ingenio.

Aquí está la consulta necesaria para mostrarte las tablas en la base de datos del Departamento de Policía. Cópiala/Pégala en la celda de abajo y ejecútala para ver las tablas disponibles para ti:

>```sql
>SELECT *
>FROM information_schema.tables
>```

In [10]:
%%sql

SELECT *
FROM information_schema.tables

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,memory,main,crime_scene_report,BASE TABLE,None,None,None,None,None,YES,NO,None,None
1,memory,main,drivers_license,BASE TABLE,None,None,None,None,None,YES,NO,None,None
2,memory,main,facebook_event_checkin,BASE TABLE,None,None,None,None,None,YES,NO,None,None
3,memory,main,get_fit_now_check_in,BASE TABLE,None,None,None,None,None,YES,NO,None,None
4,memory,main,get_fit_now_member,BASE TABLE,None,None,None,None,None,YES,NO,None,None
5,memory,main,income,BASE TABLE,None,None,None,None,None,YES,NO,None,None
6,memory,main,interview,BASE TABLE,None,None,None,None,None,YES,NO,None,None
7,memory,main,person,BASE TABLE,None,None,None,None,None,YES,NO,None,None


### Listando todas las tablas y sus columnas en la base de datos de la Policía

**Revisa la [descripción de `information_schema.columns`](https://duckdb.org/docs/sql/information_schema.html#columns)**

¡Genial! Ahora que sabes qué tablas están disponibles, es hora de averiguar las columnas que tiene cada tabla junto con el tipo de datos de cada columna. Escribe una consulta que muestre el nombre de la tabla, el nombre de la columna y el tipo de datos para cada tabla en la base de datos del Departamento de Policía utilizando la vista `information_schema.columns`. Asegúrate de que la salida esté ordenada por nombre de tabla y luego por nombre de columna (ascendente):

*Pistas opcionales - ¡intenta usarlas solo si te quedas **REALMENTE** atascado!*

<details>
<summary>Pista 1</summary>
<p>Estructura tu consulta de la misma manera que lo hicimos en la consulta anterior donde miramos `information_schema.tables`</p>
</details>

<details>
<summary>Pista 2</summary>
<p>Asegúrate de revisar el enlace para la descripción de information_schema.columns. Te dirá los nombres de las columnas que debes usar para la consulta. Recuerda, queremos el nombre de la tabla, el nombre de la columna y el tipo de datos - ¡revisa la descripción para encontrar los nombres apropiados!</p>
</details>

<details>
<summary>Pista 3</summary>
<p>No olvides usar la declaración ORDER BY para ordenar los resultados de tu búsqueda. Estamos buscando ordenar ambas columnas en orden ascendente, lo cual se puede especificar usando la palabra clave ASC, sin embargo, también es el orden de clasificación predeterminado, por lo que ASC no es necesariamente requerido.</p>
</details>

In [33]:
%%sql

SELECT DISTINCT tables.table_name, columns.column_name, columns.data_type
FROM information_schema.tables
JOIN information_schema.columns ON tables.table_schema = columns.table_schema

,table_name,column_name,data_type
0,crime_scene_report,ssn,BIGINT
1,facebook_event_checkin,ssn,BIGINT
2,get_fit_now_check_in,ssn,BIGINT
3,person,ssn,BIGINT
4,drivers_license,address_street_name,VARCHAR
5,facebook_event_checkin,address_street_name,VARCHAR
6,get_fit_now_check_in,address_street_name,VARCHAR
7,get_fit_now_member,address_street_name,VARCHAR
8,interview,address_street_name,VARCHAR
9,person,address_street_name,VARCHAR


### Entity Relationship Diagram (Optional)

After doing some detective work and finding the tables in the Police Department database, you discover that there is an ERD! This can be really helpful in your quest to find the murderer. If you like, challenge yourself (and your SQL skills) to continue on by just querying the `information_schema` as needed. Otherwise, you can click the dropdown below to reveal the ERD to help you see the tables and relationships in the Police Department database at a glance:

<details>
  <summary>Check the SQL Murder Mystery ERD</summary>
  <img src="https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/murder_mystery_schema.png?raw=1" alt="ERD for SQL Murder Mystery"/>
</details>

### Diagrama de Relaciones de Entidad (Opcional)

Después de hacer un poco de trabajo de detective y encontrar las tablas en la base de datos del Departamento de Policía, descubres que hay un ERD (Diagrama de Relaciones de Entidad). Esto puede ser realmente útil en tu búsqueda para encontrar al asesino. Si lo prefieres, desafíate a ti mismo (y tus habilidades en SQL) a continuar solo consultando el `information_schema` según sea necesario. De lo contrario, puedes hacer clic en el desplegable a continuación para revelar el ERD que te ayudará a ver las tablas y las relaciones en la base de datos del Departamento de Policía de un vistazo:

<details>
  <summary>Revisa el ERD del SQL Murder Mystery</summary>
  <img src="https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/murder_mystery_schema.png?raw=1" alt="ERD for SQL Murder Mystery"/>
</details>

In [41]:
%%sql

UsageError: %%sql is a cell magic, but the cell body is empty. Did you mean the line magic %sql (single %)?


___

## Resolviendo el Misterio (y verificando si resolviste el caso)

De aquí en adelante, usarás tus habilidades y las pistas en la base de datos del Departamento de Policía para encontrar al asesino. Cuando creas que has encontrado al asesino, vuelve a esta sección y usa la celda de abajo para verificar tu respuesta y ver si encontraste al asesino:

ejemplo:
```python
check_suspect("Adam Levine")
```

In [ ]:
check_suspect("Write the suspected murderer's full name here")

---

## Espacio de Trabajo

Usa las celdas de abajo para escribir tus consultas y trabajar en resolver el misterio. Cuando tengas un sospechoso, verifica tu respuesta usando la [celda arriba de esta](#solving-the-mystery-and-checking-if-you-cracked-the-case) y ejecuta su nombre a través de la función `check_suspect`. Si encuentras al asesino, la función te lo dirá.

No olvides usar `%%sql` en la parte superior de las celdas de `Code` que crees para que tus consultas SQL funcionen. ¡Buena suerte, detectives!

Reviso la base de satos sobre el reporte de la escena del crimen. Agrego un query donde se consulten los datos de la escena que tenga lugar en 'SQL City' específicamente.

In [40]:
%%sql
SELECT type, description
FROM crime_scene_report
WHERE city = 'SQL City';

,type,description
0,murder,REDACTED REDACTED REDACTED
1,murder,Someone killed the guard! He took an arrow to the knee!
2,theft,"A lone hunter stalks the night, firing arrows into the Darkness.\n There is no hidin..."
3,arson,"Wield the Hammer of Sol with honor, Titan, it is a thing of\n legend, both past and ..."
4,robbery,The Gjallarhorn shoulder-mounted rocket system was forged from\n the armor of Guardi...
5,bribery,"Apparently, Cayde thought it necessary to expose this extremely\n rare vegetable to ..."
6,assault,"Hamilton: Lee, do you yield? Burr: You shot him in the side! Yes he yields!"
7,assault,Report Not Found
8,murder,Security footage shows that there were 2 witnesses. The first witness lives at the last house o...


In [50]:
%%sql
SELECT *
FROM get_fit_now_check_in
WHERE check_in_date LIKE '2018%'

,membership_id,check_in_date,check_in_time,check_out_time
0,NL318,20180212,329,365
1,NL318,20170811,469,920
2,NL318,20180429,506,554
3,NL318,20180128,124,759
4,NL318,20171027,418,1019
5,NL318,20171018,608,1074
6,NL318,20171127,850,954
7,NL318,20170627,591,715
8,NL318,20180114,777,813
9,NL318,20171107,148,460


In [44]:
%%sql
SELECT *
FROM get_fit_now_member

,id,person_id,name,membership_start_date,membership_status
0,NL318,65076,Everette Koepke,20170926,gold
1,AOE21,39426,Noe Locascio,20171005,regular
2,2PN28,63823,Jeromy Heitschmidt,20180215,silver
3,0YJ24,80651,Waneta Wellard,20171206,gold
4,3A08L,32858,Mei Bianchin,20170401,silver
5,6W6I2,69510,Paris Accurso,20170613,regular
6,5Y28Y,15218,Millicent Yessios,20180205,gold
7,156A7,79701,Marcelino Leard,20180428,silver
8,A5N3S,24541,Mary Cameron,20170501,gold
9,X0643,15247,Shondra Ledlow,20170521,silver


In [45]:
%%sql
SELECT *
FROM interview

,person_id,transcript
0,28508,‘I deny it!’ said the March Hare.\n
1,63713,\n
2,86208,"way, and the whole party swam to the shore.\n"
3,35267,"lessons in here? Why, there’s hardly room for YOU, and no room at all\n"
4,33856,\n
5,82799,\n
6,66493,would be offended again.\n
7,46263,"‘So he did, so he did,’ said the Gryphon, sighing in his turn; and both\n"
8,54263,very glad that it was over at last: ‘and I do so like that curious song\n
9,62764,come to the end: then stop.’\n
